# Using GraphClient

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">Introduction</a></span></li>
        <li><span><a href="#Query" data-toc-modified-id="Query-2">Query</a></span></li>
        <li><span><a href="#Search" data-toc-modified-id="Search-3">Search</a></span></li>
        <li><span><a href="#Data-Editing" data-toc-modified-id="Data-Editing-4">Data Editing</a></span>
            <ul>
                <li><span><a href="#Adds" data-toc-modified-id="Adds-4.1">Adds</a></span></li>
                <li><span><a href="#Updates" data-toc-modified-id="Updates-4.2">Updates</a></span></li>
                <li><span><a href="#Deletes" data-toc-modified-id="Deletes-4.3">Deletes</a></span></li>
            </ul>
        </li>
        <li><span><a href="#Data-Model-Editing" data-toc-modified-id="Data-Model-Editing-5">Data Model Editing</a></span>
            <ul>
                <li><span><a href="#Types" data-toc-modified-id="Types-5.1">Types</a></span>
                    <ul>
                        <li><span><a href="#Adds" data-toc-modified-id="Adds-5.1.1">Adds</a></span></li>
                        <li><span><a href="#Updates" data-toc-modified-id="Updates-5.1.2">Updates</a></span></li>
                        <li><span><a href="#Deletes" data-toc-modified-id="Deletes-5.1.3">Deletes</a></span></li>
                    </ul>
                </li>
                <li><span><a href="#Properties" data-toc-modified-id="Properties-5.2">Properties</a></span></li>
                    <ul>
                        <li><span><a href="#Adds" data-toc-modified-id="Adds-5.2.1">Adds</a></span></li>
                        <li><span><a href="#Updates" data-toc-modified-id="Updates-5.2.2">Updates</a></span></li>
                        <li><span><a href="#Deletes" data-toc-modified-id="Deletes-5.2.3">Deletes</a></span></li>
                    </ul>
                <li><span><a href="#Search-Indexes" data-toc-modified-id="Search-Indexes-5.3">Search Indexes</a></span></li>
            </ul>
        </li>
    </ul>
</div>

## Introduction

`GraphClient` provides classes for common things like `EntityType`, `RelationshipType`, `Entity`, `Relationship`, `GraphProperty` and more. 

In [ ]:
from arcgis.gis import GIS
from arcgis.geometry import Point
from arcgis.graph import GraphClient, EntityType, RelationshipType, Entity, Relationship, GraphProperty, EntityDelete, NamedObjectTypeMask, GraphPropertyMask, SearchIndexProperties

gis = GIS("home")
new_service = gis.content.create_service(
    name="MyKnowledgeGraph",
    capabilities="Query,Editing,Create,Update,Delete",
    service_type="KnowledgeGraph"
)
graph = GraphClient(new_service.url, gis=gis)

## Query

`query` in `GraphClient` will execute a `query_streaming` and provides extra query options such as using `bind_param` to define bind parameters to be used in the query and `include_provenance` which will query for provenance records if set to `True`

In [ ]:
result = graph.query("MATCH (n) RETURN n LIMIT 5")
list(result)

## Search

`search` searches the graph for the given string on any properties of both entities or relationships by default. To search for properties on only entities or only relationships you can use the `category` parameter on `search` and set the value to either `entities`, `relationships`, or `both` (default).

In [ ]:
result = graph.search("esri")

## Data Editing

`apply_edits` consists of `adds`, `updates`, and `deletes` each of which takes a list of objects for the operation. These objects can be `Entity` or `Relationship`.

### Adds

In [ ]:
add_entity = Entity(
    type_name="Company", 
    properties={
        "name": "Esri",
        "year_established": 1969
    }
)

add_relationship = Relationship(
    type_name="WorksFor", 
    origin_entity_id="{56BCCAF0-DD03-487A-BE39-F32164714190}",
    destination_entity_id="{26BCCAF0-DD03-484A-BE39-B32164718190}",
    properties={
        "startDate": datetime(2020, 1, 6)
    }
)

graph.apply_edits(adds=[add_entity, add_relationship])

### Updates

In [ ]:
update_entity = Entity(
    type_name="Company", 
    id="{26BCCAF0-DD03-484A-BE39-B32164718190}",
    properties={
        "name": "Not Esri",
        "shape": Point(
            {
                "x":-89.86140978600001, 
                "y":38.902491682000004,
                "spatialReference": {"wkid": 4326}
            }
        )
    }
)

graph.apply_edits(updates=[update_entity])

### Deletes

In [ ]:
delete_entity = EntityDelete(
    type_name="Company", 
    ids=["{26BCCAF0-DD03-484A-BE39-B32164718190}"]
)

graph.apply_edits(deletes=[delete_entity], cascade_delete=True) # be sure to set cascade_delete to True to automatically delete relationships attached to the provided entity

## Data Model Editing

### Types

#### Adds

In [ ]:
# create the data model
graph.named_object_type_adds(
    entity_types=[
        EntityType(name="User", properties={"name": GraphProperty(name="name")}),
        EntityType(name="Item", properties={
            "id": GraphProperty(name="id"),
            "title": GraphProperty(name="title"),
            "type": GraphProperty(name="type")
        })
    ], 
    relationship_types=[
        RelationshipType(name="DependsOn"),
        RelationshipType(name="Owns")
    ]
)

#### Updates

In [ ]:
graph.named_object_type_update(
    type_name="User", 
    named_type_update=EntityType(name="User", alias="NewUser", strict=False), 
    mask=NamedObjectTypeMask(
        update_alias=True,
        update_strict=True
    )
)

#### Deletes

In [ ]:
graph.named_object_type_delete(type_name="User")

### Properties

#### Adds

In [ ]:
graph.graph_property_adds(
    type_name="User", 
    graph_properties=[
        GraphProperty(name="name"),
        Graphproperty(name="post_count", field_type="esriFieldTypeInteger"),
        GraphProperty(name="shape", field_type="esriFieldTypeGeometry", geometry_type="esriGeometryPoint")
    ]
)

#### Updates

In [ ]:
graph.graph_property_update(
    type_name="User", 
    property_name="name",
    graph_property=GraphProperty(name="name", alias="user_name", field_type="esriFieldTypeString"),
    mask=GraphPropertyMask(
        update_alias=True,
        update_field_type=True
    )
)

#### Deletes

In [ ]:
graph.graph_property_delete(type_name="User", property_name="name")

### Search Indexes

#### Adds

In [ ]:
graph.update_search_index(adds={"User": SearchIndexProperties(property_names=["name"])})

In [ ]:
# add search indexes to all text properties based on the data model
datamodel = graph.query_data_model()
for entity_type in datamodel.entity_types:
    prop_list = []
    for prop in datamodel.entity_types[entity_type].properties:
        if datamodel.entity_types[entity_type].properties[prop].fieldType == 'esriFieldTypeString':
            prop_list.append(prop)
    knowledge_graph.update_search_index(adds={entity_type: SearchIndexProperties(property_names=prop_list)})
for entity_type in datamodel.relationship_types:
    prop_list = []
    for prop in datamodel.relationship_types[entity_type].properties:
        if datamodel.relationship_types[entity_type].properties[prop].fieldType == 'esriFieldTypeString':
            prop_list.append(prop)
    knowledge_graph.update_search_index(adds={entity_type: SearchIndexProperties(property_names=prop_list)})

#### Deletes

In [ ]:
graph.update_search_index(deletes={"User": SearchIndexProperties(property_names=["name"])})